In [ ]:
import python_weather
import asyncio

from pandas.tseries.holiday import USFederalHolidayCalendar
import holidays 

When are sales best? Is it correlated with weather? Snow for the mountain? Holidays? COVID because everytone had to get outside

In [ ]:
Compare sales data with placer cell phone data. Can we find trends and do some forecasting??


In [ ]:
# categories_list = pd.read_csv('../data/raw/category-list.csv')
# inconsistent_categories = set(products['Category']).difference(categories_list)
# print(len(inconsistent_categories))

sales_csv = "../data/raw/Sales 1-3-2024.csv"
sales_category_excel = "../data/raw/Sales_By_Category_20231226_1648.xlsx"
sales_history_csv = "../data/raw/Sales/2023 - Sunlight Ski and Bike Shop.csv"
# sales_cat = pd.read_excel(sales_category_excel)

In [ ]:
async def get_weather(city):
    async with python_weather.Client(unit=python_weather.IMPERIAL) as client:
        weather = await client.get(city)
        return weather

# Assuming you have a DataFrame 'historic_data' with a 'date' column
sales['weather_type'] = None  # Initialize the new column
sales['SaleDate'] = pd.to_datetime(sales['SaleDate'])
# Loop through each row in the DataFrame
for index, row in sales.iterrows():
    date = row['SaleDate']
    
    # Convert the date to a string in the format used by the weather API
    date_str = date.strftime('%Y-%m-%d')
    
    # Use the get_weather function to fetch weather data for the date
    weather = asyncio.run(get_weather('Glenwood Springs'))  # Replace 'New York' with your city

    # Extract the weather information you are interested in (e.g., description, main, etc.)
    weather_type = weather.current.weather['description']
    
    # Assign the weather type to the corresponding row in the DataFrame
    sales.at[index, 'weather_type'] = weather_type

# Display the updated DataFrame
sales.head()

In [ ]:
def is_holiday(x): 
    usa_holidays = holidays.UnitedStates()  # Use 'UnitedStates' instead of 'country_holidays('USA')'
    if x in usa_holidays:  # Check if the date is in the list of holidays
        return 1
    else:
        return 0

sales['holidays'] = sales['SaleDate'].apply(is_holiday) 

In [ ]:
# Load data
data = products

# Define a list of stopwords
stop_words = stopwords.words('english')

# Bag of Words (BoW) method
vectorizer_bow = CountVectorizer(stop_words=stop_words)
description_bow = vectorizer_bow.fit_transform(data['Description'])

# TF-IDF (Term Frequency-Inverse Document Frequency) method
vectorizer_tfidf = TfidfVectorizer(stop_words=stop_words)
description_tfidf = vectorizer_tfidf.fit_transform(data['Description'])

# Word Embeddings (Word2Vec) method
word2vec_model = gensim.models.Word2Vec(sentences=data['Description'], vector_size=100, window=5, min_count=1, workers=4)

word_embeddings = []
for word in data['Description']:
    if word in word2vec_model.wv:
        word_embeddings.append(word2vec_model.wv[word])
    else:
        word_embeddings.append(np.zeros(100))  # If word not present, use zero vector

# Doc2Vec method
# doc2vec_model = gensim.models.Doc2Vec(vector_size=100, min_count=2, epochs=40)
# doc2vec_model.build_vocab(data['Description'])
# doc2vec_model.train(data['Description'], total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)

# TextCategorizer (spaCy) method
nlp = spacy.load("en_core_web_sm")

# Add the text categorizer component to the pipeline using its string name
nlp.add_pipe('textcat')

# Machine Learning Classification Algorithms
X_train, X_test, y_train, y_test = train_test_split(data['Description'], data['Category'], test_size=0.2, random_state=42)

# Define models
models = {
    "Logistic Regression": LogisticRegression(),
    "Decision Tree": DecisionTreeClassifier(),
    "Naive Bayes": MultinomialNB(),
    "Random Forest": RandomForestClassifier(),
    "Support Vector Machine": SVC()
}

# Evaluate each model
for name, model in models.items():
    if "Naive Bayes" in name:
        model.fit(vectorizer_tfidf.transform(X_train), y_train)
        y_pred = model.predict(vectorizer_tfidf.transform(X_test))
    else:
        model.fit(vectorizer_bow.transform(X_train), y_train)
        y_pred = model.predict(vectorizer_bow.transform(X_test))

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)

    # Calculate precision
    precision = precision_score(y_test, y_pred, average='weighted')

    # Print results
    print(f"Model: {name}")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Precision: {precision:.2f}")

    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    print(f"Confusion Matrix:\n{cm}\n")

In [ ]:
doc2vec_model = Doc2Vec(vector_size=100, min_count=2, epochs=40)

tokenized_descriptions = [description.split() for description in products['Description']]

# Convert each document (product description) to a TaggedDocument object
tagged_data = [TaggedDocument(words=words, tags=[str(i)]) for i, words in enumerate(tokenized_descriptions)]

# Now tagged_data contains a list of TaggedDocument objects

# Build vocabulary from the tagged data
doc2vec_model.build_vocab(tagged_data)

# Train the Doc2Vec model
doc2vec_model.train(tagged_data, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)

In [ ]:
# in case the libraries are not in the proper path
# import sys
# sys.executable

# !/opt/homebrew/anaconda3/bin/python -m pip install xgboost fasttext nltk spacy